# A Guide to Gradient Boosted Trees with XGBoost in Python

![](XG_Cover.jpg)

## Loading and Cleaning Data

In [1]:
# Import numpy and pandas for data wrangling

import numpy as np
import pandas as pd


In [2]:
# Data can be found at https://archive.ics.uci.edu/ml/datasets/Adult
# Load data:

train_set = pd.read_csv('adult.data', header = None)
test_set = pd.read_csv('adult.test',
                      skiprows = 1, header = None) # Make sure to skip a row for the test set


In [3]:
# Check training data for issues:

train_set.head()

0                  1       2           3   4                    5   \
0  39          State-gov   77516   Bachelors  13        Never-married   
1  50   Self-emp-not-inc   83311   Bachelors  13   Married-civ-spouse   
2  38            Private  215646     HS-grad   9             Divorced   
3  53            Private  234721        11th   7   Married-civ-spouse   
4  28            Private  338409   Bachelors  13   Married-civ-spouse   

                   6               7       8        9     10  11  12  \
0        Adm-clerical   Not-in-family   White     Male  2174   0  40   
1     Exec-managerial         Husband   White     Male     0   0  13   
2   Handlers-cleaners   Not-in-family   White     Male     0   0  40   
3   Handlers-cleaners         Husband   Black     Male     0   0  40   
4      Prof-specialty            Wife   Black   Female     0   0  40   

               13      14  
0   United-States   <=50K  
1   United-States   <=50K  
2   United-States   <=50K  
3   United-States   <=50K  
4            Cuba   <=50K

In [4]:
# Check testing data for issues:

test_set.head()

0           1       2              3   4                    5   \
0  25     Private  226802           11th   7        Never-married   
1  38     Private   89814        HS-grad   9   Married-civ-spouse   
2  28   Local-gov  336951     Assoc-acdm  12   Married-civ-spouse   
3  44     Private  160323   Some-college  10   Married-civ-spouse   
4  18           ?  103497   Some-college  10        Never-married   

                   6           7       8        9     10  11  12  \
0   Machine-op-inspct   Own-child   Black     Male     0   0  40   
1     Farming-fishing     Husband   White     Male     0   0  50   
2     Protective-serv     Husband   White     Male     0   0  40   
3   Machine-op-inspct     Husband   Black     Male  7688   0  40   
4                   ?   Own-child   White   Female     0   0  30   

               13       14  
0   United-States   <=50K.  
1   United-States   <=50K.  
2   United-States    >50K.  
3   United-States    >50K.  
4   United-States   <=50K.

In [5]:
# We don't have a column header for our data
# There seem to be some unknown values in the fifth row of the test set (the question marks) we need to deal with
# The target values have periods at the end in the test set but do not in the training set (<=50K. vs. <=50K)

In [6]:
# Add Column names to data
# Find column names at https://archive.ics.uci.edu/ml/datasets/Adult

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']

train_set.columns = col_labels
test_set.columns = col_labels

In [7]:
# How many rows are there in each dataframe  before we clean up ' ?'
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education_num     32561 non-null int64
marital_status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital_gain      32561 non-null int64
capital_loss      32561 non-null int64
hours_per_week    32561 non-null int64
native_country    32561 non-null object
wage_class        32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [8]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 15 columns):
age               16281 non-null int64
workclass         16281 non-null object
fnlwgt            16281 non-null int64
education         16281 non-null object
education_num     16281 non-null int64
marital_status    16281 non-null object
occupation        16281 non-null object
relationship      16281 non-null object
race              16281 non-null object
sex               16281 non-null object
capital_gain      16281 non-null int64
capital_loss      16281 non-null int64
hours_per_week    16281 non-null int64
native_country    16281 non-null object
wage_class        16281 non-null object
dtypes: int64(6), object(9)
memory usage: 1.9+ MB


In [9]:
# Removing unknown values (rows with ' ?'):

# Number of rows decreased from 32,561 to 30,162:

train_set.replace(' ?', np.nan).dropna().shape 


(30162, 15)

In [10]:
# Number of rows decreased from 16,281 to 15,060:

test_set.replace(' ?', np.nan).dropna().shape 

(15060, 15)

In [11]:
# Apply changes to dataset:

train_nomissing = train_set.replace(' ?', np.nan).dropna()
test_nomissing = test_set.replace(' ?', np.nan).dropna()

In [12]:
# Remove periods from incomes:

test_nomissing['wage_class'] = test_nomissing.wage_class.replace({' <=50K.': ' <=50K', ' >50K.':' >50K'})

In [13]:
# Double check that they match:

test_nomissing.wage_class.unique()

array([' <=50K', ' >50K'], dtype=object)

In [14]:
train_nomissing.wage_class.unique()

array([' <=50K', ' >50K'], dtype=object)

## Applying Ordinal Encoding to Categoricals

In other words, all of our attributes need to have numerical values, as opposed to strings or other data types

In [15]:
# Combine the train and test data to encode them:

combined_set = pd.concat([train_nomissing, test_nomissing], axis = 0) # Stacks them vertically

In [16]:
# Check which columns are not numerical:

combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
age               45222 non-null int64
workclass         45222 non-null object
fnlwgt            45222 non-null int64
education         45222 non-null object
education_num     45222 non-null int64
marital_status    45222 non-null object
occupation        45222 non-null object
relationship      45222 non-null object
race              45222 non-null object
sex               45222 non-null object
capital_gain      45222 non-null int64
capital_loss      45222 non-null int64
hours_per_week    45222 non-null int64
native_country    45222 non-null object
wage_class        45222 non-null object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [17]:
# We can use pandas Categorical codes to encode strings to ints

for feature in combined_set.columns: # Loop through all columns in the dataframe
    if combined_set[feature].dtype == 'object': # Only apply for columns with categorical strings
        combined_set[feature] = pd.Categorical(combined_set[feature]).codes # Replace strings with an integer

In [18]:
# Check the types of the columns again to ensure they're all numerical:

combined_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 16280
Data columns (total 15 columns):
age               45222 non-null int64
workclass         45222 non-null int8
fnlwgt            45222 non-null int64
education         45222 non-null int8
education_num     45222 non-null int64
marital_status    45222 non-null int8
occupation        45222 non-null int8
relationship      45222 non-null int8
race              45222 non-null int8
sex               45222 non-null int8
capital_gain      45222 non-null int64
capital_loss      45222 non-null int64
hours_per_week    45222 non-null int64
native_country    45222 non-null int8
wage_class        45222 non-null int8
dtypes: int64(6), int8(9)
memory usage: 2.8 MB


In [19]:
# Split the combined data back into two data sets:

final_train = combined_set[:train_nomissing.shape[0]] # Up to the last initial training set row
final_test = combined_set[train_nomissing.shape[0]:] # Past the last initial training set row

## Initial Model Setup and Grid Search

In [20]:
# Remove the last column from each data set, since this is what we are trying to determine

y_train = final_train.pop('wage_class')
y_test = final_test.pop('wage_class')

In [21]:
# Import XGBoost and GridSearchCV from sklearn
# The tools from the sklearn API are compatible with XGBoost :)

import xgboost as xgb
from sklearn.model_selection import GridSearchCV
# from sklearn.grid_search import GridSearchCV

#Grid Searching determines the optimal parameters to use in a data model

ModuleNotFoundError: No module named 'xgboost'

In [28]:
# We run our grid search with 5-fold cross-validation and see which parameters perform the best:

optimized_GBM.fit(final_train, y_train)

NameError: name 'optimized_GBM' is not defined

In [ ]:
# Check grid scores:

optimized_GBM.grid_scores_

In [ ]:
# We can see that the first hyperparameter combination performed best!

# Let's try optimizing some other hyperparameters now to see if we can beat a mean of 86.78% accuracy. 
# This time, we will play around with subsampling along with lowering the learning rate to see if that helps.

In [ ]:
# Set different parameters now:

cv_params = {'learning_rate': [0.1, 0.01], 'subsample': [0.7,0.8,0.9]}
ind_params = {'n_estimators': 1000, 'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth': 3, 'min_child_weight': 1}


optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1)
optimized_GBM.fit(final_train, y_train)

In [ ]:
# Check new grid scores:

optimized_GBM.grid_scores_

# We see these new cv parameters didn't bring any improvements :(

## Early stopping CV

Based on the CV testing performed earlier, we know these parameters work best:

- Learning_rate (eta) = 0.1
- Subsample, colsample_bytree = 0.8
- Max_depth = 3
- Min_child_weight = 1


In [ ]:
# We can create a DMatrix to make XGBoost faster in many iterations
# This sorts the data initially to optimize for XGBoost when it builds trees, making the algorithm more efficient. 

xgdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient

In [ ]:
# Specify the parameters we know are good:

our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':1} 

# Set up early stopping
# Stop training if result doesn't improve after 100 rounds:
cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], 
                early_stopping_rounds = 100) 

In [ ]:
# Look at our results (which are automatically saved in a pandas dataframe yay)
cv_xgb.tail(5)

In [ ]:
#Our CV test error at this number of iterations is 12.897%,  or 87.103% accuracy.
#Now that we have our best settings, let's create this as an XGBoost object model that we can reference later. 

our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)

## Plotting Feature Importance with Seaborn

In [ ]:
# Show graph in notebook:

%matplotlib inline

# Import seaborn and set font size:

import seaborn as sns
sns.set(font_scale = 1.5)

In [ ]:
# Plot graph using our object model:

xgb.plot_importance(final_gb)

We find that he 'fnlwgt' feature seems to have the most importance. fnlwgt is determined by all other characteristics. It's very complicated, but people with similar demographics should have similar weights.

Filing capital gains was also important, which makes sense given that only those with greater incomes have the ability to invest. 

Race and sex were not as important. This may be because we are just predicting a specific threshold (below/above 50,000 a year) instead of a precise income. 

## Analyzing Performance on Test Data

In [ ]:
# Now we test!!

# We create another DMatrix for test data:

testdmat = xgb.DMatrix(final_test)

In [ ]:
# Now let's use sklearn's accuracy metric to see how well we did on the test set.

from sklearn.metrics import accuracy_score

y_pred = final_gb.predict(testdmat) # Predict test data using final model
y_pred

You can see that the predict function for XGBoost outputs probabilities by default and not actual class labels. To calculate accuracy we need to convert these to a 0/1 label. We will set 0.5 probability as our threshold. 

In [ ]:
# XGBoost outputs probabilities by default but we can change them to booleans to check the accuracy of the data:

y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred

In [ ]:
# Now we can calculate our accuracy: 

accuracy_score(y_pred, y_test), 1-accuracy_score(y_pred, y_test)

Our final accuracy is 86.94%, or a 13.05% error rate. Not bad at all!